In [ ]:
# download data
! wget https://huggingface.co/datasets/anonmsr/girt-instruct/resolve/main/train.csv
! wget https://huggingface.co/datasets/anonmsr/girt-instruct/resolve/main/val.csv
! wget https://huggingface.co/datasets/anonmsr/girt-instruct/resolve/main/test.csv

In [ ]:
from datasets import load_dataset
from datasets import DatasetDict

# Load dataset from the hub
my_dataset_train = load_dataset('csv', data_files='train.csv', split="train", cache_dir = './cache')
my_dataset_train = my_dataset_train.shuffle(seed=42)
my_dataset_val = load_dataset('csv', data_files='val.csv', split="train", cache_dir = './cache')
my_dataset_train = my_dataset_train.shuffle(seed=42)
my_dataset_test = load_dataset('csv', data_files='test.csv', split="train", cache_dir = './cache')


my_dataset = DatasetDict({
    'train': my_dataset_train,
    'test': my_dataset_test,
    'val': my_dataset_val
})

In [ ]:
my_dataset['train'] = my_dataset['train'].remove_columns(['_type'])
my_dataset['val'] = my_dataset['val'].remove_columns(['_type'])
my_dataset['test'] = my_dataset['test'].remove_columns(['_type'])


In [ ]:
print(f"Train dataset size: {len(my_dataset['train'])}")
print(f"Val dataset size: {len(my_dataset['val'])}")
print(f"Test dataset size: {len(my_dataset['test'])}")

In [ ]:
from random import randrange

sample = my_dataset['train'][randrange(len(my_dataset["train"]))]
print(f"{sample['instruction']}\n---------------")
print(f"{sample['output']}\n---------------")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="t5-base"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir = './cache')

In [ ]:
special_tokens_dict = {'additional_special_tokens': ['<|Repo_Name|>', '<|Email|>', '<|URL|>', '<|Code|>', '<|Image|>', '<|MASK|>', '<|EMPTY|>']}

# check the number of special tokens
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

from tokenizers import AddedToken
num_added_toks+=tokenizer.add_tokens(AddedToken("\n", normalized=False))
num_added_toks+=tokenizer.add_tokens(AddedToken("<!---", normalized=False))
num_added_toks+=tokenizer.add_tokens(AddedToken("-->", normalized=False))

print('We have added', num_added_toks, 'tokens')

In [ ]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([my_dataset["train"], my_dataset["test"], my_dataset["val"]]).map(lambda x: tokenizer(x["instruction"], truncation=True), batched=True, remove_columns=["output", "instruction"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([my_dataset["train"], my_dataset["test"], my_dataset["val"]]).map(lambda x: tokenizer(x["output"], truncation=True), batched=True, remove_columns=["output", "instruction"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")


In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["instruction"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["output"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = my_dataset.map(preprocess_function, batched=True, remove_columns=["output", "instruction"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="t5-base"


# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, cache_dir = './cache')


In [ ]:
# change max length to 300 for evaluation part - in average IRTs have a length of 300
model.config.max_length = 300

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric_meteor = evaluate.load("meteor")
metric_rouge = evaluate.load("rouge")
metric_bleu = evaluate.load("bleu")


# helper function to postprocess text
def postprocess_text_rouge(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels


# helper function to postprocess text
def postprocess_text_bleu(preds, labels):
    preds = [pred.strip() for pred in preds]
    # list of list based on documentation
    labels = [[label.strip()] for label in labels]

    return preds, labels

# helper function to postprocess text
def postprocess_text_meteor(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge
    decoded_preds_rouge, decoded_labels_rouge = postprocess_text_rouge(decoded_preds, decoded_labels)
    result_rouge = metric_rouge.compute(predictions=decoded_preds_rouge, references=decoded_labels_rouge, use_stemmer=True)
    
    # Meteor
    decoded_preds_meteor, decoded_labels_meteor = postprocess_text_meteor(decoded_preds, decoded_labels)
    result_meteor = metric_meteor.compute(predictions=decoded_preds_meteor, references=decoded_labels_meteor)
    
    # Bleu
    decoded_preds_bleu, decoded_labels_bleu = postprocess_text_bleu(decoded_preds, decoded_labels)
    result_bleu = metric_bleu.compute(predictions=decoded_preds_bleu, references=decoded_labels_bleu)

    # norm
    result = {**result_rouge,**result_meteor, **result_bleu}
    
    # mean length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)


    # Define the file path
    file_path = 'evaluate_dict.txt'

    # Iterate through the list of dictionaries
    with open(file_path, 'a') as file:
        file.write(str(result) + '\n')

    return result


In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
# Hugging Face repository id
repository_id = f"girt-{model_id.split('/')[1]}"
output_dir_custom = f'./trainer_save/{repository_id}'

In [ ]:
import collections 
import collections.abc
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments



# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir_custom,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=30,
    # logging & evaluation strategies
    logging_dir=f"./trainer_log/{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=10,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    compute_metrics=compute_metrics,
)


In [ ]:
# empty cache & gb
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# train
trainer.train()

In [ ]:
# save final model
trainer.save_model("trainer_save/final")
tokenizer.save_pretrained("trainer_save/final/tokenizer")